In [2]:
import pandas as pd
import numpy as np
import vectorbt as vbt
from datetime import datetime
import os 

In [13]:
# Prepare data
start = '2019-01-01 UTC'  # crypto is in UTC
end = datetime.utcnow()
dir_name = '/Users/andre/Documents/Python/trading_clone_singapore_DO/trading_singapore_digitalocean/vectorBT'
os.makedirs(dir_name, exist_ok=True)
file_name = 'btc_price_1d.pkl'
joined = os.path.join(dir_name, file_name)

if os.path.exists(joined):
    btc_price = pd.read_pickle(joined)
    print('File exitst already')
else:
    btc_price = vbt.CCXTData.download(exchange = 'kucoin',symbols = 'BTC/USDT',timeframe='1d', start=start, end=end).get()
    btc_price.to_pickle(joined)
    print(btc_price)
    
btc_price.head()

# eth_price = vbt.YFData.download('ETH-USD', start=start, end=end).get('Close')
# comb_price = btc_price.vbt.concat(eth_price,
#     keys=pd.Index(['BTC', 'ETH'], name='symbol'))
# comb_price.vbt.drop_levels(-1, inplace=True)
# comb_price

File exitst already


,Open,High,Low,Close,Volume
Open time,,,,,
2019-01-01 00:00:00+00:00,3700.170853,3805.841739,3645.525763,3799.644687,97.900396
2019-01-02 00:00:00+00:00,3799.644687,3880.257733,3750.000001,3856.504318,159.232564
2019-01-03 00:00:00+00:00,3857.633097,3862.600830,3728.917225,3765.999334,110.163702
2019-01-04 00:00:00+00:00,3767.854300,3821.195343,3706.000000,3791.461260,92.251254
2019-01-05 00:00:00+00:00,3789.000000,3838.674540,3756.410862,3772.156039,93.872993


# first succesful try

In [14]:
import numpy as np
import pandas as pd
import vectorbt as vbt
from numba import njit
from vectorbt.portfolio.enums import Direction, SizeType

# Create sample price data
btc_price2 = btc_price.head(100).copy()
close_price = btc_price2['Close']


# Define order size (could be a strategy-specific calculation)
# Assume btc_price['Close'], btc_price['High'], btc_price['Low'], and btc_price['Open'] are Series
close = btc_price['Close'].to_numpy()
high = btc_price['High'].to_numpy()
low = btc_price['Low'].to_numpy()
open_ = btc_price['Open'].to_numpy()
btc_price2 = btc_price.to_numpy()

# Numba-compiled order function
@njit
def order_func_nb(c,high,low,open_):

    close_price = c.close[c.i, c.col]
    print("INDEX",c.i)
    print("COL",c.col)
    print("high",high[c.i])
    print('open',open_[c.i])
    print("clsoe",close_price)
    print('low',low[c.i])
    
    #print("lAST_OIDX",c.last_oidx)
    #print("position_record",c.pos_record_now)
    print('position size :',c.position_now)
    if c.position_now > 0:
        print("POSITION IS LONG")
        sl_price = c.pos_record_now[4]* 0.95
        tp_price = c.pos_record_now[4] * 1.05
    print("TP_PRICE",tp_price)
    print("SL_PRICE",sl_price)    
    print()

    
    if (c.position_now == 0) & (c.i != 0) :
        return vbt.portfolio.nb.order_nb(
            size=1.0,  # Fixed size of 1 share/contract
            price=close_price,  # Current closing price
            size_type=SizeType.Amount,  # Specify size type
            direction=Direction.Both  # Long-only trading
        )
    elif (c.position_now > 0) & (sl_price > low[c.i]):
            value = vbt.portfolio.nb.order_nb(
            size=-1,  # Close position
            price=sl_price,
            size_type=SizeType.Amount,
            direction=Direction.LongOnly)  # Long-only trading
            sl_price = 0
            print('stoploss hit')
            print("return value ",value)
            print()

            return value
    elif (c.position_now > 0) & (tp_price < high[c.i]):
            value = vbt.portfolio.nb.order_nb(
            size=-1,  # Close position
            price=tp_price,
            size_type=SizeType.Amount,
            direction=Direction.LongOnly)

            print('tp hit')
            print("return value ",value)
            print()
            return value


    
    return vbt.portfolio.enums.NoOrder

        

# Create portfolio
pf = vbt.Portfolio.from_order_func(
    close_price,           # Price DataFrame
    order_func_nb,
    high,
    low,
    open_,    # Order function
    init_cash=10000  # Initial cash balance
)

# Display some portfolio performance metrics
print("Total Return:", pf.total_return())
print("\nOrder Records:")

pf.orders.records_readable
# Optional: Plot equity curve
pf.plot().show()

INDEX 0
COL 0
high 3805.841739
open 3700.170853
clsoe 3799.644687
low 3645.525763
position size : 0.0
TP_PRICE 0.0
SL_PRICE 0.0

INDEX 1
COL 0
high 3880.257733
open 3799.644687
clsoe 3856.504318
low 3750.000001
position size : 0.0
TP_PRICE 0.0
SL_PRICE 0.0

INDEX 2
COL 0
high 3862.60083
open 3857.633097
clsoe 3765.999334
low 3728.917225
position size : 1.0
POSITION IS LONG
TP_PRICE 4049.3295339
SL_PRICE 3663.6791020999995

INDEX 3
COL 0
high 3821.195343
open 3767.8543
clsoe 3791.46126
low 3706.0
position size : 1.0
POSITION IS LONG
TP_PRICE 4049.3295339
SL_PRICE 3663.6791020999995

INDEX 4
COL 0
high 3838.67454
open 3789.0
clsoe 3772.156039
low 3756.410862
position size : 1.0
POSITION IS LONG
TP_PRICE 4049.3295339
SL_PRICE 3663.6791020999995

INDEX 5
COL 0
high 4023.0
open 3775.703265
clsoe 3988.471816
low 3746.584563
position size : 1.0
POSITION IS LONG
TP_PRICE 4049.3295339
SL_PRICE 3663.6791020999995

INDEX 6
COL 0
high 4017.71888
open 3984.865903
clsoe 3972.264189
low 3927.799365
p

## ma cross overall trend prediction

In [33]:

def ma_cross_slow_indicator(close_price,window_slow,window_mid,window_fast):
    ema4 = vbt.MA.run(close_price, window=window_slow).ma.to_numpy()
    ema5 = vbt.MA.run(close_price, window=window_mid).ma.to_numpy()
    ema6 = vbt.MA.run(close_price, window=window_fast).ma.to_numpy()
    up = (ema4 > ema5) & (ema5 > ema6)
    down = (ema4 < ema5) & (ema5 < ema6)
    trend = np.where(up,1,np.where(down,-1,0))

    return trend


ma_cross_slow = vbt.IndicatorFactory(class_name='MA_Cross_slow',
                                     short_name='ma_c_slow',
                                     input_names=['Close'],
                                     param_names=['ema4', 'ema5', 'ema6'],
                                     output_names=['ma_cross_slow']
                                     ).from_apply_func(ma_cross_slow_indicator,ema4= 20, ema5=40,ema6=70)

# window_slow = [180,200]
# window_mid = [40,50]
# window_fast = [20,30]

window_slow = 180
window_mid = 40
window_fast = 20

trend = ma_cross_slow.run(btc_price['Close'],ema4= window_fast, ema5=window_mid,ema6=window_slow,param_product=True)
trend.ma_cross_slow




Open time
2019-01-01 00:00:00+00:00    0
2019-01-02 00:00:00+00:00    0
2019-01-03 00:00:00+00:00    0
2019-01-04 00:00:00+00:00    0
2019-01-05 00:00:00+00:00    0
                            ..
2024-12-13 00:00:00+00:00    1
2024-12-14 00:00:00+00:00    1
2024-12-15 00:00:00+00:00    1
2024-12-16 00:00:00+00:00    1
2024-12-17 00:00:00+00:00    1
Freq: D, Name: (20, 40, 180, Close), Length: 2178, dtype: int64

In [57]:
import vectorbt as vbt
import numpy as np

def ma_cross_rsi(close_price, ma1_window, ma2_window, ma3_window, ma4_window, ma5_window, rsi_buy_threshold, rsi_sell_threshold):
    ma1 = vbt.MA.run(close_price, window=ma1_window).ma.to_numpy()
    ma2 = vbt.MA.run(close_price, window=ma2_window).ma.to_numpy()
    ma3 = vbt.MA.run(close_price, window=ma3_window).ma.to_numpy()
    ma4 = vbt.MA.run(close_price, window=ma4_window).ma.to_numpy()
    ma5 = vbt.MA.run(close_price, window=ma5_window).ma.to_numpy()
    slow_trend_up = (ma4 > ma5)
    slow_trend_down = (ma4 < ma5)

    short_trend_up = (ma1 > ma2) & (ma2 > ma3)
    short_trend_undecided = (ma1 < ma2) & (ma2 > ma3)
    short_trend_down = (ma1 < ma2) & (ma2 < ma3)

    rsi = vbt.RSI.run(close_price, window=14).rsi.to_numpy()
    rsi_buy = rsi > rsi_buy_threshold
    rsi_sell = rsi < rsi_sell_threshold
    
    signal_buy = slow_trend_up & short_trend_up & rsi_buy
    signal_sell = slow_trend_down & short_trend_down & rsi_sell

    no_caution_long = slow_trend_up & short_trend_up
    caution_long1 = slow_trend_up & short_trend_undecided 
    caution_long2 = slow_trend_up & short_trend_down

    no_caution_short = slow_trend_down & short_trend_down
    caution_short1 = slow_trend_down & short_trend_undecided
    caution_short2 = slow_trend_down & short_trend_up

    conditions = [
        signal_buy,
        no_caution_long,
        caution_long1,
        caution_long2,
        signal_sell,
        no_caution_short,
        caution_short1,
        caution_short2
    ]

    values = [
        100,
        75,
        50,
        25,
        -100,
        -75,
        -50,
        -25
    ]


    # Use np.select to apply the conditions
    signal = np.select(conditions, values, default=0)
  
    # signal = np.where(signal_buy, 100, np.where(signal_sell, -100, sl_adjust_signal))

    return signal

ma_cross_slow = vbt.IndicatorFactory(
    class_name='MA_Cross_rsi',
    short_name='ma_c_slow',
    input_names=['Close'],
    param_names=['ma1', 'ma2', 'ma3', 'ma4', 'ma5', 'rsi_buy_threshold', 'rsi_sell_threshold'],
    output_names=['ma_cross_rsi']
).from_apply_func(ma_cross_rsi, ma1=20, ma2=30, ma3=40, ma4=125, ma5=150, rsi_buy_threshold=70, rsi_sell_threshold=30)

# Define window sizes and RSI thresholds
ma_trend_fast = 25
ma_trend_mid = 50
ma_trend_slow = 70

ma_slow_trend1 = 125
ma_slow_trend_2 = 150

rsi_buy_threshold = 70
rsi_sell_threshold = 30

# Run the indicator
trend = ma_cross_slow.run(
    btc_price['Close'],
    ma1=ma_trend_fast,
    ma2=ma_trend_mid,
    ma3=ma_trend_slow,
    ma4=ma_slow_trend1,
    ma5=ma_slow_trend_2,
    rsi_buy_threshold=rsi_buy_threshold,
    rsi_sell_threshold=rsi_sell_threshold,
    param_product=True
)
trend.ma_cross_rsi.head(150)

Open time
2019-01-01 00:00:00+00:00     0
2019-01-02 00:00:00+00:00     0
2019-01-03 00:00:00+00:00     0
2019-01-04 00:00:00+00:00     0
2019-01-05 00:00:00+00:00     0
                             ..
2019-05-26 00:00:00+00:00     0
2019-05-27 00:00:00+00:00     0
2019-05-28 00:00:00+00:00     0
2019-05-29 00:00:00+00:00     0
2019-05-30 00:00:00+00:00    75
Freq: D, Name: (25, 50, 70, 125, 150, 70, 30, Close), Length: 150, dtype: int64

In [59]:
import vectorbt as vbt
import numpy as np

def ma_cross_rsi(close_price, ma1_window, ma2_window, ma3_window, ma4_window, ma5_window, rsi_buy_threshold, rsi_sell_threshold):
    # Calculate moving averages
    ma1 = vbt.MA.run(close_price, window=ma1_window).ma.to_numpy()
    ma2 = vbt.MA.run(close_price, window=ma2_window).ma.to_numpy()
    ma3 = vbt.MA.run(close_price, window=ma3_window).ma.to_numpy()
    ma4 = vbt.MA.run(close_price, window=ma4_window).ma.to_numpy()
    ma5 = vbt.MA.run(close_price, window=ma5_window).ma.to_numpy()
    
    # Determine slow trend direction
    slow_trend_up = (ma4 > ma5)
    slow_trend_down = (ma4 < ma5)

    # Determine short trend direction
    short_trend_up = (ma1 > ma2) & (ma2 > ma3)
    short_trend_undecided = (ma1 < ma2) & (ma2 > ma3)
    short_trend_down = (ma1 < ma2) & (ma2 < ma3)

    # Calculate RSI and determine buy/sell signals
    rsi = vbt.RSI.run(close_price, window=14).rsi.to_numpy()
    rsi_buy = rsi > rsi_buy_threshold
    rsi_sell = rsi < rsi_sell_threshold
    
    # Generate buy and sell signals
    signal_buy = slow_trend_up & short_trend_up & rsi_buy
    signal_sell = slow_trend_down & short_trend_down & rsi_sell

    # Define caution signals for long and short positions
    no_caution_long = slow_trend_up & short_trend_up
    caution_long1 = slow_trend_up & short_trend_undecided 
    caution_long2 = slow_trend_up & short_trend_down

    no_caution_short = slow_trend_down & short_trend_down
    caution_short1 = slow_trend_down & short_trend_undecided
    caution_short2 = slow_trend_down & short_trend_up

    # Define conditions and corresponding values for signals
    conditions = [
        signal_buy,
        no_caution_long,
        caution_long1,
        caution_long2,
        signal_sell,
        no_caution_short,
        caution_short1,
        caution_short2
    ]

    values = [
        100,  # Strong buy signal (slow trend up, short trend up, and RSI above the buy threshold)
        75,   # No caution long (slow trend up and short trend up)
        50,   # Caution long 1 (slow trend up and short trend undecided)
        25,   # Caution long 2 (slow trend up and short trend down)
        -100, # Strong sell signal (slow trend down, short trend down, and RSI below the sell threshold)
        -75,  # No caution short (slow trend down and short trend down)
        -50,  # Caution short 1 (slow trend down and short trend undecided)
        -25   # Caution short 2 (slow trend down and short trend up)
    ]

    # Use np.select to apply the conditions and assign the corresponding values
    signal = np.select(conditions, values, default=0)  # 0: No signal (default value when none of the conditions are met)

    return signal

# Create the indicator factory
ma_cross_slow = vbt.IndicatorFactory(
    class_name='MA_Cross_rsi',
    short_name='ma_c_slow',
    input_names=['Close'],
    param_names=['ma1', 'ma2', 'ma3', 'ma4', 'ma5', 'rsi_buy_threshold', 'rsi_sell_threshold'],
    output_names=['ma_cross_rsi']
).from_apply_func(ma_cross_rsi, ma1=20, ma2=30, ma3=40, ma4=125, ma5=150, rsi_buy_threshold=70, rsi_sell_threshold=30)

# Define window sizes and RSI thresholds
ma_trend_fast = 25
ma_trend_mid = 50
ma_trend_slow = 70

ma_slow_trend1 = 125
ma_slow_trend_2 = 150

rsi_buy_threshold = 70
rsi_sell_threshold = 30

# Run the indicator
trend = ma_cross_slow.run(
    btc_price['Close'],
    ma1=ma_trend_fast,
    ma2=ma_trend_mid,
    ma3=ma_trend_slow,
    ma4=ma_slow_trend1,
    ma5=ma_slow_trend_2,
    rsi_buy_threshold=rsi_buy_threshold,
    rsi_sell_threshold=rsi_sell_threshold,
    param_product=True
)

# Display the first 150 rows of the signal output
trend.ma_cross_rsi.head(150)

Open time
2019-01-01 00:00:00+00:00     0
2019-01-02 00:00:00+00:00     0
2019-01-03 00:00:00+00:00     0
2019-01-04 00:00:00+00:00     0
2019-01-05 00:00:00+00:00     0
                             ..
2019-05-26 00:00:00+00:00     0
2019-05-27 00:00:00+00:00     0
2019-05-28 00:00:00+00:00     0
2019-05-29 00:00:00+00:00     0
2019-05-30 00:00:00+00:00    75
Freq: D, Name: (25, 50, 70, 125, 150, 70, 30, Close), Length: 150, dtype: int64

In [32]:
import numpy as np
import pandas as pd
import vectorbt as vbt
from numba import njit
from vectorbt.portfolio.enums import Direction, SizeType

# Create sample price data
btc_price2 = btc_price.copy()
close_price = btc_price2['Close']

# Assume btc_price['Close'], btc_price['High'], btc_price['Low'], and btc_price['Open'] are Series
close = btc_price['Close'].to_numpy()
high = btc_price['High'].to_numpy()
low = btc_price['Low'].to_numpy()
open_ = btc_price['Open'].to_numpy()
btc_price2 = btc_price.to_numpy()
entries = trend.ma_cross_slow.to_numpy()

# Create an array to store SL prices
sl_prices = np.full(close.shape[0], np.nan)  # Use a 1D array



# Numba-compiled order function
@njit
def order_func_nb(c, high, low, open_, entries, sl_prices):
    close_price = c.close[c.i, c.col]
    print("INDEX", c.i)
    print("COL", c.col)
    print("high", high[c.i])
    print('open', open_[c.i])
    print("close", close_price)
    print('low', low[c.i])
    print('position size :', c.position_now)
    print('cash:', c.cash_now)
    print('entries:', entries[c.i])
    print()

# if in position 
    if (c.position_now > 0):
        # Check if SL is hit
        if low[c.i-1] <= sl_prices[c.i]:
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,  # Close position
                price=sl_prices[c.i],
                size_type=SizeType.Amount,
                direction=Direction.LongOnly)

            print('sl hit at:', c.i, sl_prices[c.i])
            print("return value ", value)
            return value
        
        # check condition to adjust sl 
        # find lowest low in the last 5 bars
        # set sl n% lower than the lowest low
        # if new sl is higher than the current sl update
        # if new sl is lower than the current sl do nothing

        if c.i >= 5:  # Ensure there are enough bars to look back
            lowest_low = np.min(low[c.i-5:c.i])  # Find the lowest low in the last 5 bars
            new_sl_price = lowest_low * 0.90  # Set SL n% lower than the lowest low
            if new_sl_price > sl_prices[c.i]:
                if (entries[c.i] == 1):
                    print('sl update before',c.i,sl_prices[c.i])
                    sl_prices[:]= close_price * 0.90 # Calculate SL price
                    print('sl update after at index',c.i,sl_prices[c.i])
                if (entries[c.i] == 0):            
                    print('sl update before',c.i,sl_prices[c.i])
                    sl_prices[:]= close_price * 0.95 # Calculate SL price
                    print('sl update after at index',c.i,sl_prices[c.i])


    # if not in position search for position to enter
    elif (c.position_now == 0) & (c.i != 0):
        if entries[c.i] == 1:
            sl_price = close_price * 0.90
            order = vbt.portfolio.nb.order_nb(
                size=0.1,  # Adjusted order size
                price=close_price,  # Current closing price
                size_type=SizeType.Percent,  # Specify size type
                direction=Direction.LongOnly,  # Long-only trading
                fees=0.0,  # No fees
                slippage=0.0,  # No slippage
                allow_partial=False,  # Do not allow partial fills
                raise_reject=True  # Raise an error if the order is rejected
            )
            sl_prices[:] = sl_price 
            print('sl to put in array price:', sl_price)
            print('sl array',sl_prices[1])   # Save the SL price using a single index
            print('long order', c.i,order)
            print()
            return order
    


    return vbt.portfolio.enums.NoOrder


# Create portfolio
pf = vbt.Portfolio.from_order_func(
    close_price,           # Price DataFrame
    order_func_nb,
    high,
    low,
    open_,
    entries,    # Order function
    sl_prices,  # Pass the SL prices array
    init_cash=100000,  # Initial cash balance
)

# Display some portfolio performance metrics
print("Total Return:", pf.total_return())
print("\nOrder Records:")

pf.orders.records_readable
# Optional: Plot equity curve
pf.plot().show()



INDEX 0
COL 0
high 3805.841739
open 3700.170853
close 3799.644687
low 3645.525763
position size : 0.0
cash: 100000.0
entries: 0

INDEX 1
COL 0
high 3880.257733
open 3799.644687
close 3856.504318
low 3750.000001
position size : 0.0
cash: 100000.0
entries: 0

INDEX 2
COL 0
high 3862.60083
open 3857.633097
close 3765.999334
low 3728.917225
position size : 0.0
cash: 100000.0
entries: 0

INDEX 3
COL 0
high 3821.195343
open 3767.8543
close 3791.46126
low 3706.0
position size : 0.0
cash: 100000.0
entries: 0

INDEX 4
COL 0
high 3838.67454
open 3789.0
close 3772.156039
low 3756.410862
position size : 0.0
cash: 100000.0
entries: 0

INDEX 5
COL 0
high 4023.0
open 3775.703265
close 3988.471816
low 3746.584563
position size : 0.0
cash: 100000.0
entries: 0

INDEX 6
COL 0
high 4017.71888
open 3984.865903
close 3972.264189
low 3927.799365
position size : 0.0
cash: 100000.0
entries: 0

INDEX 7
COL 0
high 4061.280555
open 3977.721738
close 3956.635321
low 3925.0
position size : 0.0
cash: 100000.0
entrie